In [1]:
import hail as hl

In [2]:
from Killing_RP import *

### 1- row key rsid

In [24]:
gwas = hl.import_table('snp_gwas.txt', types={'POS': hl.tint})
reference = hl.import_table('snp_ref.txt',  types={'position': hl.tint}).add_index()

2018-04-30 15:43:22 Hail: INFO: Reading table with no type imputation
  Loading column 'SNP' as type 'str' (type not specified)
  Loading column 'CHR' as type 'str' (type not specified)
  Loading column 'POS' as type 'int32' (user-specified)
  Loading column 'A1' as type 'str' (type not specified)
  Loading column 'A2' as type 'str' (type not specified)
  Loading column 'Case' as type 'str' (type not specified)

2018-04-30 15:43:22 Hail: INFO: Reading table with no type imputation
  Loading column 'rsid' as type 'str' (type not specified)
  Loading column 'chrom' as type 'str' (type not specified)
  Loading column 'position' as type 'int32' (user-specified)
  Loading column 'alt' as type 'str' (type not specified)
  Loading column 'ref' as type 'str' (type not specified)
  Loading column 'Case' as type 'str' (type not specified)



In [25]:
gwas = gwas.key_by('SNP')
gwas = gwas.annotate(alleles=[gwas.A1,gwas.A2])

reference = reference.key_by('rsid')
reference = reference.annotate(locus=hl.struct(contig=reference.chrom, 
                                               position=reference.position), 
                               alleles=[reference.ref, reference.alt])

In [7]:
gwas.show()

+-------+-----+-------+-----+-----+------------------+------------+
| SNP   | CHR |   POS | A1  | A2  | Case             | alleles    |
+-------+-----+-------+-----+-----+------------------+------------+
| str   | str | int32 | str | str | str              | array<str> |
+-------+-----+-------+-----+-----+------------------+------------+
| snp1  | 1   |     1 | A   | G   | snp;direct;nsa   | ["A","G"]  |
| snp2  | 1   |     2 | A   | G   | snp;swap;nsa     | ["A","G"]  |
| snp3  | 1   |     3 | A   | G   | snp;flip;nsa     | ["A","G"]  |
| snp4  | 1   |     4 | A   | G   | snp;swapflip;nsa | ["A","G"]  |
| snp5  | 1   |     5 | T   | A   | snp;direct;sa    | ["T","A"]  |
| snp6  | 1   |     6 | T   | A   | snp;swap;sa      | ["T","A"]  |
| snp7  | 1   |     7 | A   | AT  | indel;direct     | ["A","AT"] |
| snp8  | 1   |     8 | A   | AT  | indel;swap       | ["A","AT"] |
| snp9  | 1   |     9 | A   | C   | snp;multi        | ["A","C"]  |
| snp10 | 1   |    10 | A   | G   | nomatch     

In [8]:
reference.show()

+------+-------+----------+-----+-----+------------------+-------+--------------+
| rsid | chrom | position | alt | ref | Case             |   idx | locus.contig |
+------+-------+----------+-----+-----+------------------+-------+--------------+
| str  | str   | str      | str | str | str              | int64 | str          |
+------+-------+----------+-----+-----+------------------+-------+--------------+
| snp1 | 1     | 1        | A   | G   | snp;direct;nsa   |     0 | 1            |
| snp2 | 1     | 2        | G   | A   | snp;swap;nsa     |     1 | 1            |
| snp3 | 1     | 3        | T   | C   | snp;flip;nsa     |     2 | 1            |
| snp4 | 1     | 4        | C   | T   | snp;swapflip;nsa |     3 | 1            |
| snp5 | 1     | 5        | T   | A   | snp;direct;sa    |     4 | 1            |
| snp6 | 1     | 6        | A   | T   | snp;swap;sa      |     5 | 1            |
| snp7 | 1     | 7        | A   | AT  | indel;direct     |     6 | 1            |
| snp8 | 1     |

In [5]:
# matched = match_variants(gwas_row_key, gwas_alleles, reference_row_key, reference_alleles)
matched = match_variants(gwas.SNP, gwas.alleles, reference.rsid, reference.alleles)

2018-04-30 15:29:50 Hail: INFO: Ordering unsorted dataset with network shuffle
2018-04-30 15:29:50 Hail: INFO: Coerced sorted dataset
2018-04-30 15:29:51 Hail: INFO: Ordering unsorted dataset with network shuffle


In [19]:
m = matched.drop('CHR', 'POS').explode('matches')
m = m.transmute(**m.matches)
m = m.drop(m.chrom, m.position, m.Case, m.idx, m.locus, m.alleles)
m.show(width=100, n=20)

+-------+-----+-----+-----+-----+---------------------------------------------------------+
| SNP   | A1  | A2  | alt | ref | match_alleles                                           |
+-------+-----+-----+-----+-----+---------------------------------------------------------+
| str   | str | str | str | str | array<struct{swap: bool, flip: bool}>                   |
+-------+-----+-----+-----+-----+---------------------------------------------------------+
| snp1  | A   | G   | A   | G   | [{"swap":false,"flip":false}]                           |
| snp12 | A   | C   | A   | G   | []                                                      |
| snp2  | A   | G   | G   | A   | [{"swap":true,"flip":false}]                            |
| snp3  | A   | G   | T   | C   | [{"swap":false,"flip":true}]                            |
| snp4  | A   | G   | C   | T   | [{"swap":true,"flip":true}]                             |
| snp5  | T   | A   | T   | A   | [{"swap":true,"flip":true},{"swap":false,"flip

### 2-row key locus

In [26]:
gwas = gwas.annotate(locus = hl.struct(contig=gwas.CHR, 
                                       position=gwas.POS))
gwas = gwas.key_by('locus')

reference = reference.key_by('locus')

In [27]:
# matched = match_variants(gwas_row_key, gwas_alleles, reference_row_key, reference_alleles)
matched_by_locus = match_variants(gwas.locus, gwas.alleles, reference.locus, reference.alleles)

2018-04-30 15:43:39 Hail: INFO: Coerced sorted dataset
2018-04-30 15:43:39 Hail: INFO: Coerced sorted dataset
2018-04-30 15:43:39 Hail: INFO: Coerced sorted dataset


In [32]:
m = matched_by_locus.drop('CHR', 'POS').explode('matches')
m = m.transmute(**m.matches)
m = m.drop(m.chrom, m.position, m.Case, m.idx, m.locus, m.alleles, m.rsid)
m.show(width=100, n=20)

+-------+-----+-----+-----+-----+---------------------------------------------------------+
| SNP   | A1  | A2  | alt | ref | match_alleles                                           |
+-------+-----+-----+-----+-----+---------------------------------------------------------+
| str   | str | str | str | str | array<struct{swap: bool, flip: bool}>                   |
+-------+-----+-----+-----+-----+---------------------------------------------------------+
| snp1  | A   | G   | A   | G   | [{"swap":false,"flip":false}]                           |
| snp2  | A   | G   | G   | A   | [{"swap":true,"flip":false}]                            |
| snp3  | A   | G   | T   | C   | [{"swap":false,"flip":true}]                            |
| snp4  | A   | G   | C   | T   | [{"swap":true,"flip":true}]                             |
| snp5  | T   | A   | T   | A   | [{"swap":true,"flip":true},{"swap":false,"flip":false}] |
| snp6  | T   | A   | A   | T   | [{"swap":true,"flip":false},{"swap":false,"fli